# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [37]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

loan_amnt  int_rate  installment home_ownership  annual_inc  \
0    10500.0    0.1719       375.35           RENT     66000.0   
1    25000.0    0.2000       929.09       MORTGAGE    105000.0   
2    20000.0    0.2000       529.88       MORTGAGE     56000.0   
3    10000.0    0.1640       353.55           RENT     92000.0   
4    22000.0    0.1474       520.39       MORTGAGE     52000.0   

  verification_status   issue_d loan_status pymnt_plan    dti  ...  \
0     Source Verified  Mar-2019    low_risk          n  27.24  ...   
1            Verified  Mar-2019    low_risk          n  20.23  ...   
2            Verified  Mar-2019    low_risk          n  24.26  ...   
3            Verified  Mar-2019    low_risk          n  31.44  ...   
4        Not Verified  Mar-2019    low_risk          n  18.76  ...   

   pct_tl_nvr_dlq  percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  \
0            85.7             100.0                   0.0        0.0   
1            91.2              50.0                   1.0        0.0   
2            66.7              50.0                   0.0        0.0   
3           100.0              50.0                   1.0        0.0   
4           100.0               0.0                   0.0        0.0   

   tot_hi_cred_lim  total_bal_ex_mort total_bc_limit  \
0          65687.0            38199.0         2000.0   
1         271427.0            60641.0        41200.0   
2          60644.0            45684.0         7500.0   
3          99506.0            68784.0        19700.0   
4         219750.0            25919.0        27600.0   

   total_il_high_credit_limit  hardship_flag  debt_settlement_flag  
0                     61987.0              N                     N  
1                     49197.0              N                     N  
2                     43144.0              N                     N  
3                     76506.0              N                     N  
4                     20000.0              N                     N  

[5 rows x 86 columns]

## Split the Data into Training and Testing

In [42]:
# Create our features
X = df.copy()
X.drop(columns = "loan_status", axis = 1, inplace = True)
X = pd.get_dummies(X)
X
# Create our target
y = df.loan_status
y

0        low_risk
1        low_risk
2        low_risk
3        low_risk
4        low_risk
           ...   
68812    low_risk
68813    low_risk
68814    low_risk
68815    low_risk
68816    low_risk
Name: loan_status, Length: 68817, dtype: object

In [ ]:
X.describe()

In [44]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [49]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [51]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [53]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [55]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [74]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators = 64, random_state = 1)
rf_model = rf_model.fit(X_train_scaled, y_train)


In [75]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.767146993581491

In [76]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   68,    33],
       [ 2377, 14727]], dtype=int64)

In [77]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.67      0.86      0.05      0.76      0.57       101
   low_risk       1.00      0.86      0.67      0.92      0.76      0.59     17104

avg / total       0.99      0.86      0.67      0.92      0.76      0.59     17205



In [78]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)

[(0.06961684339214233, 'total_rec_prncp'),
 (0.0605996901545109, 'total_pymnt_inv'),
 (0.05982367085134305, 'total_rec_int'),
 (0.05595792422387815, 'total_pymnt'),
 (0.050419041730121884, 'last_pymnt_amnt'),
 (0.02907985575395111, 'int_rate'),
 (0.02247793147622121, 'issue_d_Jan-2019'),
 (0.02073940942755754, 'installment'),
 (0.017755659823368332, 'out_prncp_inv'),
 (0.016959135129782442, 'dti'),
 (0.016254630150368744, 'out_prncp'),
 (0.015386684101303403, 'total_bal_ex_mort'),
 (0.015313086305407671, 'mths_since_rcnt_il'),
 (0.015023033781180318, 'mo_sin_old_rev_tl_op'),
 (0.014827945332017603, 'tot_hi_cred_lim'),
 (0.014759597552976445, 'mths_since_recent_inq'),
 (0.014750002551426892, 'max_bal_bc'),
 (0.014341239784947854, 'total_il_high_credit_limit'),
 (0.014226632528878881, 'issue_d_Mar-2019'),
 (0.014165661531547503, 'bc_util'),
 (0.013679020184961322, 'annual_inc'),
 (0.013560722617331464, 'pct_tl_nvr_dlq'),
 (0.013559641727819302, 'total_bc_limit'),
 (0.013118514386049341, 

### Easy Ensemble Classifier

In [68]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier 

rf_model1 = EasyEnsembleClassifier(n_estimators = 64, random_state = 1)
rf_model1 = rf_model.fit(X_train_scaled, y_train)

In [69]:
# Calculated the balanced accuracy score
y_pred = rf_model1.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9313969750576554

In [70]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   93,     8],
       [  992, 16112]], dtype=int64)

In [71]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy ensemble classifier

2. Which model had the best recall score?

    Easy ensemble

3. Which model had the best geometric mean score?

    Easy ensemble

4. What are the top three features?

    total_rec_prncp & total_pymnt_inv & total_rec_int